### Approximate the peak 2D function using a neural netExtra required packages:
- matplotlib

In [ ]:
import gurobipy as gp
import numpy as np
import torch
from gurobipy import GRB
from matplotlib import cm
from matplotlib import pyplot as plt
from sklearn import metrics
from gurobi_ml import add_predictor_constr

In [ ]:
def peak2d(xx, yy):
    return (
        3 * (1 - xx) ** 2.0 * np.exp(-(xx**2) - (yy + 1) ** 2)
        - 10 * (xx / 5 - xx**4 - yy**5) * np.exp(-(xx**2) - yy**2)
        - 1 / 3 * np.exp(-((xx + 1) ** 2) - yy**2)
    )

In [ ]:
x = torch.arange(-2, 2, 0.01)
y = torch.arange(-2, 2, 0.01)
x1, x2 = torch.meshgrid(x, y, indexing="ij")
z = peak2d(x1, x2)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# Plot the surface.
surf = ax.plot_surface(x1, x2, z, cmap=cm.coolwarm, linewidth=0.01, antialiased=False)
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
X = torch.cat([x1.ravel().reshape(-1, 1), x2.ravel().reshape(-1, 1)], axis=1)
y = z.ravel()

In [ ]:
hs = 30
# Define a simple sequential network
model = torch.nn.Sequential(
    torch.nn.Linear(X.shape[1], hs),
    torch.nn.ReLU(),
    torch.nn.Linear(hs, hs),
    torch.nn.ReLU(),
    torch.nn.Linear(hs, 1),
)

In [ ]:
# Construct our loss function and an Optimizer.
criterion = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
for t in range(1000):
    # Zero gradients
    optimizer.zero_grad()
    # Forward pass: Compute predicted y by passing x to the model
    pred = model(X)
    # Compute and print loss
    loss = criterion(pred, y.reshape(-1, 1))
    if t % 100 == 0:
        print(f"iteration {t} loss: {loss.item()}")

    if loss.item() < 1e-4:
        break
    # Backward pass in network to compute gradients
    loss.backward()
    # Update weights
    optimizer.step()
else:
    print(f"iteration {t} loss: {loss.item()}")

In [ ]:
X_test = torch.rand((100, 2)) * 2 - 1

In [ ]:
metrics.r2_score(peak2d(X_test[:, 0], X_test[:, 1]), model(X_test).detach().numpy())

In [ ]:
metrics.max_error(peak2d(X_test[:, 0], X_test[:, 1]), model(X_test).detach().numpy())

In [ ]:
model(X).min()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# Plot the surface.
surf = ax.plot_surface(
    x1,
    x2,
    model(X).detach().numpy().reshape(x1.shape),
    cmap=cm.coolwarm,
    linewidth=0.01,
    antialiased=False,
)
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

### Do the optimization model

In [ ]:
# Start with classical part of the model
m = gp.Model()

x = m.addMVar((1, 2), lb=-2, ub=2, name="x")
y = m.addMVar(1, lb=-GRB.INFINITY, name="y")

m.setObjective(y.sum(), gp.GRB.MINIMIZE)

# Add network trained by pytorch to Gurobi model to predict y from x
nn2gurobi = add_predictor_constr(m, model, x, y)

### Finally optimize it

In [ ]:
m.Params.TimeLimit = 10
m.Params.MIPGap = 0.1

In [ ]:
m.optimize()

### Look at the solution

In [ ]:
x.X

In [ ]:
peak2d(x.X[0, 0], x.X[0, 1])

In [ ]:
y.X

Copyright © 2023 Gurobi Optimization, LLC